# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)




## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an Indian restaurant in London, UK.

Since there are lots of restaurants in London, we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Indian restaurants in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on these criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.


## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
*	number of existing restaurants in the neighborhood (any type of restaurant)
*	number of and distance to Indian restaurants in the neighborhood if any
*	distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
*	centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Foursquare API reverse geocoding**
*	number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
*	coordinate of London center will be obtained using **Foursquare API geocoding** of well-known London location (Trafalgar Square).

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around London city center.

Let's first find the latitude & longitude of London city center, using specific, well known address and Foursquare API geocoding.

In [15]:
# Before we get the data and start exploring it, let's download all the dependencies that we will need.
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


According to "http://shadyoldlady.com/location.php?loc=463" the exact center of London is marked by a plaque in the Church of St. Martin's-in-the-Fields overlooking Trafalgar Square. So we are going to look for the coordinates of Trafalgar Square.

In [29]:
address = 'Trafalgar Square, London, UK'

geolocator = Nominatim(user_agent="uk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
london_center=[latitude, longitude]


print('The geographical coordinate of London City center are {}.' .format(london_center))


The geographical coordinate of London City center are [51.508037, -0.12804941070724718].


Next, we are going to find the locations of all Indian restaurants within 6000 m from the City center.

In [85]:
# The code was removed by Watson Studio for sharing.

Your credentails are hidden


In [18]:
search_query = 'Indian'
radius = 6000
print(search_query + ' .... OK!')

Indian .... OK!


In [84]:
# The code was removed by Watson Studio for sharing.

In [20]:
results = requests.get(url).json()


#### We will get the relevant part of JSON and transform it into a *pandas* dataframe

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4b4f8d20f964a520170b27e3,6 New Row,GB,London,United Kingdom,St Martins Lane,360,"[6 New Row (St Martins Lane), London, Greater ...","[{'label': 'display', 'lat': 51.51109845704908...",51.511098,-0.126345,WC2N 5LH,Greater London,The Grand Indian,v-1587851851,NaN
1,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4bc20095abf4952177c0c193,3-7 Endell St,GB,London,United Kingdom,NaN,723,"[3-7 Endell St, London, Greater London, WC2H 9...","[{'label': 'display', 'lat': 51.51391986616573...",51.513920,-0.123602,WC2H 9RA,Greater London,Royals Indian Cuisine,v-1587851851,NaN
2,"[{'id': '4bf58dd8d48988d1ee931735', 'name': 'H...",False,4ae59bebf964a5200ca121e3,41 Fitzroy Sq,GB,London,United Kingdom,Grafton Way,1787,"[41 Fitzroy Sq (Grafton Way), London, Greater ...","[{'label': 'display', 'lat': 51.52260147680140...",51.522601,-0.138919,W1T 6AQ,Greater London,Indian YMCA,v-1587851851,NaN
3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,51782522e4b0b3a24d336e1a,115-117 Drummond Street,GB,NaN,United Kingdom,NaN,2215,"[115-117 Drummond Street, NW1 2HL, United King...","[{'label': 'display', 'lat': 51.52718539358642...",51.527185,-0.136773,NW1 2HL,NaN,Indian Spice Shop,v-1587851851,NaN
4,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4ae4e41ef964a520299f21e3,18 Rupert St.,GB,Soho,United Kingdom,near Shaftesbury Ave.,450,"[18 Rupert St. (near Shaftesbury Ave.), Soho, ...","[{'label': 'display', 'lat': 51.51087923308819...",51.510879,-0.132673,W1D 6DE,Greater London,Indian Brasserie,v-1587851851,NaN


Define information of interest and filter dataframe¶


In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Grand Indian,Indian Restaurant,6 New Row,GB,London,United Kingdom,St Martins Lane,360,"[6 New Row (St Martins Lane), London, Greater ...","[{'label': 'display', 'lat': 51.51109845704908...",51.511098,-0.126345,WC2N 5LH,Greater London,4b4f8d20f964a520170b27e3
1,Royals Indian Cuisine,Indian Restaurant,3-7 Endell St,GB,London,United Kingdom,NaN,723,"[3-7 Endell St, London, Greater London, WC2H 9...","[{'label': 'display', 'lat': 51.51391986616573...",51.513920,-0.123602,WC2H 9RA,Greater London,4bc20095abf4952177c0c193
2,Indian YMCA,Hostel,41 Fitzroy Sq,GB,London,United Kingdom,Grafton Way,1787,"[41 Fitzroy Sq (Grafton Way), London, Greater ...","[{'label': 'display', 'lat': 51.52260147680140...",51.522601,-0.138919,W1T 6AQ,Greater London,4ae59bebf964a5200ca121e3
3,Indian Spice Shop,Grocery Store,115-117 Drummond Street,GB,NaN,United Kingdom,NaN,2215,"[115-117 Drummond Street, NW1 2HL, United King...","[{'label': 'display', 'lat': 51.52718539358642...",51.527185,-0.136773,NW1 2HL,NaN,51782522e4b0b3a24d336e1a
4,Indian Brasserie,Indian Restaurant,18 Rupert St.,GB,Soho,United Kingdom,near Shaftesbury Ave.,450,"[18 Rupert St. (near Shaftesbury Ave.), Soho, ...","[{'label': 'display', 'lat': 51.51087923308819...",51.510879,-0.132673,W1D 6DE,Greater London,4ae4e41ef964a520299f21e3
5,Saffron Summer Indian Restuarant,Restaurant,"4 Ace Parade,Hook Road",GB,Surrey,United Kingdom,NaN,213,"[4 Ace Parade,Hook Road, Surrey, KT9 1DR, Unit...","[{'label': 'display', 'lat': 51.50951705106139...",51.509517,-0.126096,KT9 1DR,Surrey,5be9954b58002c002c560949
6,Professional Indian insulin pen supplier,Business Service,8 Main Rd,GB,London,United Kingdom,NaN,225,"[8 Main Rd, London, Greater London, E1 7AE, Un...","[{'label': 'display', 'lat': 51.50871574849058...",51.508716,-0.124980,E1 7AE,Greater London,5e8ccafc51cdd8000874c1da
7,Indian Accent,Indian Restaurant,16 Albemarle St,GB,London,United Kingdom,NaN,980,"[16 Albemarle St, London, Greater London, W1S ...","[{'label': 'display', 'lat': 51.50924380249302...",51.509244,-0.142068,W1S 4HW,Greater London,5a33002167a9fe7d943df123
8,Shad Indian Restaurant,Indian Restaurant,96 Tooley St,GB,London,United Kingdom,NaN,3159,"[96 Tooley St, Southwark, Greater London, SE1 ...","[{'label': 'display', 'lat': 51.50459129552639...",51.504591,-0.082787,SE1 2TH,Greater London,4f36e716e4b0fc67a8b07d36
9,Indian Visa Application Centre,Office,142-148 Goswell Rd.,GB,Islington,United Kingdom,NaN,2754,"[142-148 Goswell Rd., Islington, Greater Londo...","[{'label': 'display', 'lat': 51.52525425937069...",51.525254,-0.099497,EC1V 7DU,Greater London,4b62e421f964a52082572ae3


Let's visualize the Indian restaurants that are nearby

In [24]:
dataframe_filtered.name

0                              The Grand Indian
1                         Royals Indian Cuisine
2                                   Indian YMCA
3                             Indian Spice Shop
4                              Indian Brasserie
5              Saffron Summer Indian Restuarant
6      Professional Indian insulin pen supplier
7                                 Indian Accent
8                        Shad Indian Restaurant
9                Indian Visa Application Centre
10                      Truly Indian Restaurant
11                            Indian Singles UK
12                     Shapur Indian Restaurant
13                                  Indian City
14                           Indian Street Food
15                          Maida Indian Eatery
16                                 Indian Diner
17                      Indian YMCA Gandhi Hall
18                        KK Private Indian Bar
19                                Indian Lounge
20                               Indian 

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Trafalgar Square

# add a red circle marker to represent the Trafalgar Square
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Trafalgar Square',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Indian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

It is obvious that the collected data contains not only Indian restaurants so, we will try different approach.

Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Alexanderplatz. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [31]:
#pip install shapely
#!conda install -c conda-forge shapely --yes 

import shapely.geometry

#!pip install pyproj
#!conda install -c conda-forge pyproj --yes 

import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('London center longitude={}, latitude={}'.format(london_center[1], london_center[0]))
x, y = lonlat_to_xy(london_center[1], london_center[0])
print('London center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('London center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
London center longitude=-0.12804941070724718, latitude=51.508037
London center UTM X=-547023.4456756007, Y=5815641.585630685
London center longitude=-0.1280494107072397, latitude=51.508037


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [47]:
london_center_x, london_center_y = lonlat_to_xy(london_center[1], london_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = london_center_x - 6000
x_step = 600
y_min = london_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(london_center_x, london_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [48]:
map_london = folium.Map(location=london_center, zoom_start=13)
folium.Marker(london_center, popup='Trafalgar Square').add_to(map_london)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_london)
    #folium.Marker([lat, lon]).add_to(map_london)
map_london

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Indian restaurant' category, as we need info on Indian restaurants in the neighborhood.

In [75]:
# Category IDs corresponding to Indian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4bf58dd8d48988d10f941735' # 'Root' category for all food-related venues

Indian_restaurant_categories = ['54135bf5e4b08f3d2429dfe5','54135bf5e4b08f3d2429dff3','54135bf5e4b08f3d2429dff5',
                                 '54135bf5e4b08f3d2429dfe2','54135bf5e4b08f3d2429dff2','54135bf5e4b08f3d2429dfe1',
                                 '54135bf5e4b08f3d2429dfe3','54135bf5e4b08f3d2429dfe8','54135bf5e4b08f3d2429dfe9',
                                 '54135bf5e4b08f3d2429dfe6','54135bf5e4b08f3d2429dfdf','54135bf5e4b08f3d2429dfe4',
                                 '54135bf5e4b08f3d2429dfe7','54135bf5e4b08f3d2429dfea','54135bf5e4b08f3d2429dfeb',
                                 '54135bf5e4b08f3d2429dfed','54135bf5e4b08f3d2429dfee','54135bf5e4b08f3d2429dff4',
                                 '54135bf5e4b08f3d2429dfe0','54135bf5e4b08f3d2429dfdd','54135bf5e4b08f3d2429dff6',
                                 '54135bf5e4b08f3d2429dfef','54135bf5e4b08f3d2429dff0','54135bf5e4b08f3d2429dff1',
                                 '54135bf5e4b08f3d2429dfde','54135bf5e4b08f3d2429dfec']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180604'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [78]:
restaurants, Indian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
print(' done.')

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.
 done.


In [77]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found 
#Indian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    Indian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_Indian = is_restaurant(venue_categories, specific_filter=Indian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_Indian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_Indian:
                    Indian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, Indian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
Indian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('Indian_restaurants_350.pkl', 'rb') as f:
        Indian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, Indian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('Indian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(Indian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [79]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Indiann restaurants:', len(Indian_restaurants))
print('Percentage of Indiann restaurants: {:.2f}%'.format(len(Indian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 378
Total number of Indiann restaurants: 6
Percentage of Indiann restaurants: 1.59%
Average number of restaurants in neighborhood: 0.9725274725274725


In [80]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4d3dd4ccfb4c54816592e014', 'Curry Paradise', 51.45505037594604, -0.11942077746729092, 'Brixton Hill (Josephine Ave.), Brixton, Greater London, SW2 1AA, United Kingdom', 253, False, -547664.9786953497, 5809672.8137498535)
('4bdb36d92a3a0f471ef6adb6', 'Maharani', 51.46281083930036, -0.13511455622693902, '117-119 Clapham High Street, London, Greater London, SW4 7TB, United Kingdom', 326, False, -548565.1730618092, 5810757.239396268)
('5af99aa331fd14002c9587b4', 'Kricket', 51.462492, -0.112637, 'London, Greater London, SW9 8JL, United Kingdom', 300, False, -547024.2302867518, 5810394.722170714)
('556b057d498e61c8e74bd149', "Baba G's", 51.46347481755735, -0.11235810045205079, 'Pop Brixton, United Kingdom', 260, False, -546982.1215036643, 5810499.047809861)
('4ae2be59f964a520658f21e3', 'Ganapati', 51.469633991195614, -0.07292398157248463, '38 Holly Grove, Peckham, Greater London, SE15 5DF, United Kingdom', 253, False, -544122.5248518176, 5810

In [81]:
print('List of Indian restaurants')
print('---------------------------')
for r in list(Indian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(Indian_restaurants))

List of Indian restaurants
---------------------------
('5447a90d498e49ed234f6d81', 'شيزان', 51.498804, -0.166896, 'SW7 1ES (16-22 Cheval Place), London, Greater London, United Kingdom', 128, True, -549912.5431895091, 5815189.910551606)
('583c344181907921f51d0740', 'Tandoor Chop House', 51.50919205694273, -0.12563829447742592, '8 Adelaide Street, Charing Cross, Greater London, WC2N 4HZ, United Kingdom', 139, True, -546830.5801462998, 5815733.84914238)
('4ad0972ff964a520a0d820e3', 'Tayyabs', 51.5172403, -0.0634757, '89 Fieldgate St (Myrdle St), Whitechapel, Greater London, E1 1JU, United Kingdom', 172, True, -542365.9309710006, 5815718.285673337)
('5a3fcde232b61d131ddbaa9f', 'Saravanaa Bhavan', 51.510256242420176, -0.12842933215746546, '17 Charing Cross Rd, London, Greater London, WC2H 0EP, United Kingdom', 267, True, -546997.8009963958, 5815891.837959662)
('5b8ed86a04d1ae0039452d61', 'Little Kolkata', 51.514363, -0.124064, '51-53 Shelton St, London, Greater London, WC2H 9JU, United Kin

In [82]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 
Restaurants around location 102: 
Restaurants around location 103: Pimlico Spice
Restaurants around location 104: Pimlico Tandoori, Millbank Spice
Restaurants around location 105: 
Restaurants around location 106: 
Restaurants around location 107: 
Restaurants around location 108: Chatkhara, Castle Tandoori
Restaurants around location 109: 
Restaurants around location 110: Tower Tandoori


Let's now see all the collected restaurants in our area of interest on map, and let's also show Indian restaurants in different color.

In [83]:
map_London = folium.Map(location=london_center, zoom_start=13)
folium.Marker(london_center, popup='Trafalgar Square').add_to(map_London)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_indian = res[6]
    color = 'red' if is_indian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_London)
map_London

Looking good. So now we have all the restaurants in area within few kilometers from Trafalgar Square, and we know which ones are Indian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Indian restaurant!